In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
data = pd.read_csv("./input/movies_metadata.csv")

data.head(2)

/var/folders/p6/vz07lhpj0gq1j8hfrm0y5jxr0000gn/T/ipykernel_9511/3932221392.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./input/movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [23]:
targets = ["title", "overview"]

data[targets].isnull().sum()

title         6
overview    954
dtype: int64

In [24]:
data["title"] = data["title"].fillna("null")
data["overview"] = data["overview"].fillna("")
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(data["overview"])

tfidf_matrix.shape # 45466개의 영화 중 75827개 단어 출현

(45466, 75827)

In [25]:
cosine_sim = cosine_similarity(tfidf_matrix)

cosine_sim.shape

(45466, 45466)

In [26]:
title_to_idx = dict(zip(data["title"], data.index))

title_to_idx["Jumanji"]

1

In [30]:
def get_recommendations(title, cosine_sim):
    idx = title_to_idx[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0: 11]
    movie_indices = [idx for idx, _ in sim_scores]
    res = data["title"].iloc[movie_indices]
    return res

In [31]:
get_recommendations("The Dark Knight Rises", cosine_sim)

18252                                The Dark Knight Rises
12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object